In [0]:
SET use_cached_result = false;
SELECT  COUNT(DISTINCT cs_order_number) AS `order count`
        ,SUM(cs_ext_ship_cost) AS `total shipping cost`
        ,SUM(cs_net_profit) AS `total net profit`
FROM workspace.tpcds_sf1000_iceberg.catalog_sales cs1
,       workspace.tpcds_sf1000_iceberg.date_dim
,       workspace.tpcds_sf1000_iceberg.customer_address
,       workspace.tpcds_sf1000_iceberg.call_center
WHERE   d_date BETWEEN '2001-4-01' AND (
            CAST('2001-4-01' AS DATE) + interval'60'day
)
AND     cs1.cs_ship_date_sk = d_date_sk
AND     cs1.cs_ship_addr_sk = ca_address_sk
AND     ca_state = 'FL'
AND     cs1.cs_call_center_sk = cc_call_center_sk
AND     cc_county IN ('Wadena County','Richland County','Bronx County','Marshall County','Luce County')
AND     
exists(
            SELECT  *
            FROM workspace.tpcds_sf1000_iceberg.catalog_sales cs2
            WHERE   cs1.cs_order_number = cs2.cs_order_number
            AND     cs1.cs_warehouse_sk <> cs2.cs_warehouse_sk
        ) 
AND     NOT 
exists(
            SELECT  *
            FROM workspace.tpcds_sf1000_iceberg.catalog_returns cr1
            WHERE   cs1.cs_order_number = cr1.cr_order_number
        ) 
ORDER BY COUNT(DISTINCT cs_order_number)
LIMIT   100
;